In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '40_router_suite_workflow_solution'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Solution · Router Suite Workflow
This solution mirrors the lab by preparing the synthetic router export and
executing the guarded modules end-to-end.


In [ ]:
from pathlib import Path
import json
import tarfile
from forensic.modules.router.env import RouterEnvModule
from forensic.modules.router.extract import RouterExtractModule
from forensic.modules.router.manifest import RouterManifestModule
from forensic.modules.router.summarize import RouterSummarizeModule
from forensic.modules.router.common import load_router_defaults

CASE_DIR = LAB_ROOT / 'case_router_suite'
CASE_DIR.mkdir(parents=True, exist_ok=True)
PIPELINE_TS = _ts()
MODULE_DEFAULTS = {
    'env': load_router_defaults('env'),
    'extract': load_router_defaults('extract'),
    'manifest': load_router_defaults('manifest'),
    'summary': load_router_defaults('summary'),
}

EXPORT_DIR = LAB_ROOT / 'inputs' / 'router_export'
EXPORT_DIR.mkdir(parents=True, exist_ok=True)
devices = [
    {'mac': 'AA:BB:CC:DD:EE:01', 'ip': '192.168.0.10', 'hostname': 'workstation-a'},
    {'mac': 'AA:BB:CC:DD:EE:02', 'ip': '192.168.0.20', 'hostname': 'nas-backup'},
]
portforwards = [
    {'name': 'ssh-admin', 'protocol': 'tcp', 'external_port': 2222, 'internal_ip': '192.168.0.10', 'internal_port': 22},
    {'name': 'https-nas', 'protocol': 'tcp', 'external_port': 8443, 'internal_ip': '192.168.0.20', 'internal_port': 443},
]
ddns_lines = [
    'provider: secure-router-dns',
    'hostname: blue-gateway.example.net',
    'last_update: 2024-03-01T12:05:00Z',
]
event_log = [
    '2024-03-01T12:00:01Z login ok user=admin src=203.0.113.42',
    '2024-03-01T12:03:12Z firewall alert drop proto=TCP dst=198.51.100.23:443 reason=policy',
    '2024-03-01T12:04:45Z ddns update success host=blue-gateway.example.net',
]
ui_artifacts = {
    'alerts': [
        {'ts': '2024-03-01T12:03:12Z', 'severity': 'warning', 'message': 'Outbound policy blocked unusual HTTPS beacon'},
        {'ts': '2024-03-01T12:04:45Z', 'severity': 'info', 'message': 'Dynamic DNS update completed'},
    ],
    'dashboard': {'widgets': ['alerts', 'uptime', 'ddns_status']},
}

csv_write_rows_sorted(devices, EXPORT_DIR / 'devices.csv', header=['mac', 'ip', 'hostname'])
csv_write_rows_sorted(
    portforwards,
    EXPORT_DIR / 'portforwards.csv',
    header=['name', 'protocol', 'external_port', 'internal_ip', 'internal_port'],
)
(EXPORT_DIR / 'ddns.txt').write_text('\n'.join(ddns_lines) + '\n', encoding='utf-8')
(EXPORT_DIR / 'eventlog.txt').write_text('\n'.join(event_log) + '\n', encoding='utf-8')
json_dump_sorted(ui_artifacts, EXPORT_DIR / 'ui_artifacts.json')

archive_path = LAB_ROOT / 'inputs' / f'{PIPELINE_TS}_router_export.tar.gz'
with tarfile.open(archive_path, 'w:gz') as archive:
    for item in sorted(EXPORT_DIR.iterdir()):
        archive.add(item, arcname=item.name)


In [ ]:
env_module = RouterEnvModule(CASE_DIR, MODULE_DEFAULTS['env'])
env_result = env_module.run(None, CASE_DIR, {'dry_run': False, 'timestamp': PIPELINE_TS})

extract_module = RouterExtractModule(CASE_DIR, MODULE_DEFAULTS['extract'])
extract_result = extract_module.run(
    None, CASE_DIR, {'input': EXPORT_DIR, 'dry_run': False, 'timestamp': PIPELINE_TS}
)
extract_outputs = extract_result.data['outputs']
extract_root = Path(extract_outputs[0]).parent

manifest_module = RouterManifestModule(CASE_DIR, MODULE_DEFAULTS['manifest'])
manifest_result = manifest_module.run(
    None, CASE_DIR, {'source': extract_root, 'dry_run': False, 'timestamp': PIPELINE_TS}
)
manifest_path = Path(manifest_result.data['output'])

summary_module = RouterSummarizeModule(CASE_DIR, MODULE_DEFAULTS['summary'])
summary_result = summary_module.run(
    None, CASE_DIR, {'source': extract_root, 'dry_run': False, 'timestamp': PIPELINE_TS}
)
summary_path = Path(summary_result.data['output'])

pipeline_report = {
    'env_status': env_result.status,
    'extract_outputs': extract_outputs,
    'manifest_output': str(manifest_path),
    'summary_output': str(summary_path),
}
json_dump_sorted(pipeline_report, CASE_DIR / f'{PIPELINE_TS}_pipeline_report.json')
pipeline_report


In [ ]:
ui_payload = json.load((EXPORT_DIR / 'ui_artifacts.json').open('r', encoding='utf-8'))
manifest_payload = json.load(manifest_path.open('r', encoding='utf-8'))
manifest_hash = manifest_result.artifacts[0] if manifest_result.artifacts else {}
summary_preview = preview(summary_path)

inspection = {
    'alerts': ui_payload['alerts'],
    'manifest_entries': manifest_payload['files'][:3],
    'manifest_sha256': manifest_hash.get('sha256'),
    'manifest_size': manifest_hash.get('size'),
    'summary_head': summary_preview.split('\n')[:8],
}
json_dump_sorted(inspection, CASE_DIR / f'{PIPELINE_TS}_inspection.json')
inspection


In [ ]:
expected = [
    EXPORT_DIR / 'devices.csv',
    EXPORT_DIR / 'portforwards.csv',
    EXPORT_DIR / 'ddns.txt',
    EXPORT_DIR / 'eventlog.txt',
    EXPORT_DIR / 'ui_artifacts.json',
    manifest_path,
    summary_path,
]
for path in expected:
    assert Path(path).exists(), f'Missing artefact: {path}'

router_artifacts = {
    'ui_artifacts': str(EXPORT_DIR / 'ui_artifacts.json'),
    'manifest': str(manifest_path),
    'summary': str(summary_path),
}
json_dump_sorted(router_artifacts, CASE_DIR / f'{PIPELINE_TS}_artefacts.json')
router_artifacts
